In [8]:
import os
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import datetime
from datetime import date
from datetime import timedelta
from datetime import datetime
from loguru import logger
from google.cloud import bigquery
from google.oauth2 import service_account

# Get data form openweathermap API to BigQuery

In [ ]:
home_dir = os.path.abspath(os.path.dirname(__file__))
# BigQuery parameter
big_query_key_path = "{}/keys/xxxxxxxx.json".format(home_dir)
project_id = 'project_name'
dataset='home_weather'
table='krakow_current'
table_id=f"{project_id}.{dataset}.{table}"

def get_client_and_table(dataset):
    key_path = big_query_key_path
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

    client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
    return client, credentials

client, credentials = get_client_and_table(dataset=dataset)

In [ ]:
def datajson(data, i):
    doc={
        'lat': data.get('lat'), 
        'lon': data.get('lon'),
        'timezone': data.get('timezone'),
        'timezone_offset': data.get('timezone_offset'),
        'sunrise': data.get('current').get('sunrise'),
        'sunset': data.get('current').get('sunset'),
        'uvi': data.get('current').get('uvi'),
        'dt': data.get('hourly')[i].get('dt'),
        'temp': data.get('hourly')[i].get('temp'),
        'feels_like': data.get('hourly')[i].get('feels_like'),
        'pressure': data.get('hourly')[i].get('pressure'),
        'humidity': data.get('hourly')[i].get('humidity'),
        'dew_point': data.get('hourly')[i].get('dew_point'),
        'clouds': data.get('hourly')[i].get('clouds'),
        'visibility': data.get('hourly')[i].get('visibility'),
        'wind_speed': data.get('hourly')[i].get('wind_speed'),
        'wind_deg': data.get('hourly')[i].get('wind_deg'),
        'wind_gust': data.get('hourly')[i].get('wind_gust'),
        'ID': data.get('hourly')[i].get('weather')[0].get('id'),
        'Main': data.get('hourly')[i].get('weather')[0].get('main'),
        'Description': data.get('hourly')[i].get('weather')[0].get('description'),
        'year': time.gmtime(data.get('hourly')[i].get('dt')+data.get('timezone_offset'))[0],
        'month': time.gmtime(data.get('hourly')[i].get('dt')+data.get('timezone_offset'))[1],
        'day': time.gmtime(data.get('hourly')[i].get('dt')+data.get('timezone_offset'))[2],
        'hour': time.gmtime(data.get('hourly')[i].get('dt')+data.get('timezone_offset'))[3],
        'minute': time.gmtime(data.get('hourly')[i].get('dt'))[4],
        'sec': time.gmtime(data.get('hourly')[i].get('dt'))[5],
        'wday': time.gmtime(data.get('hourly')[i].get('dt')+data.get('timezone_offset'))[6],
        'yday': time.gmtime(data.get('hourly')[i].get('dt')+data.get('timezone_offset'))[7]    
         
    }
    return doc

In [ ]:
value = date.today()- timedelta(days=1)
dtt=date.timetuple(value) # date to tuple struct
unix_epoch = int(time.mktime(dtt)) # struct to Unix epoch 
logger.info("Yesterday: {}".format(value))
logger.info("Unix epoch: {}".format(unix_epoch))

try:        
    url = 'http://api.openweathermap.org/data/2.5/onecall/timemachine'
    param = {
            "lat":50.06465,
            "lon":19.94498,
            "dt":unix_epoch,
            "units":'metric',
            "appid":'XXXXXXXXXXXXXXX'
            }
    response = requests.get(url,param)
    data = response.json()
    w_list=[]
    for i in range(0, len(data.get('hourly'))): 
        ww=datajson(data,i)
        w_list.append(ww)
    df_w=pd.DataFrame(w_list)
        
    # Save to BigQuery
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField('lat', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('lon', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('dt', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('temp', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('feels_like', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('dew_point', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('pressure', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('humidity', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('clouds', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('visibility', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('wind_speed', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('wind_deg', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('wind_gust', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('year', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('month', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('day', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('hour', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('minute', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('sec', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('wday', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('yday', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('timezone_offset', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('sunrise', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('sunset', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('uvi', 'FLOAT', 'NULLABLE'),
            bigquery.SchemaField('ID', 'INTEGER', 'NULLABLE'),
            bigquery.SchemaField('Main', 'STRING', 'NULLABLE'),
            bigquery.SchemaField('Description', 'STRING', 'NULLABLE'),
            bigquery.SchemaField('timezone', 'STRING', 'NULLABLE'),
        ])  
        job = client.load_table_from_dataframe(df_w, table_id, job_config=job_config) 
        job.result()  # Wait for the job to complete.
        table = client.get_table(table_id)  # Make an API request.
        logger.info("Krakow --- Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
   
    except Exception:
        logger.info('What is it?')  
else:
    logger.info('Data is OK')  